# LSTMモデルに対して加えられる改善の調査レポート

## 背景

丸野さんの論文、及び以前の再現実験によって、LSTMが一定機械学習による人間計算可能なパスワードへの攻撃の制度を上昇させることがわかった。一方で、精度の向上幅はまだ限られており、攻撃が可能であるとは言い難い。そこで、今回分析しようとしている教師データが時系列データでないことなどに着目して、よりよい予測が可能になるのではないかと考えている。そのための調べものをしたのでまとめている。

## LSTM/RNNの双方向化

LSTMを含むRNNの手法は時系列データの解析に用いられる手法である。一方で、人間計算可能なパスワードは時系列に依存しないデータであり、LSTMの記憶機構は主に関数内の定数mappingの記憶に活用されていると考えられる。RNNは一般的には時系列に対して進行する方向にのみ記憶を行うが、双方向RNNと呼ばれる手法を用いることで双方向に記憶を保存することが可能である。双方向LSTMは次の要素から構成されている。

- Forward LSTM
- backward LSTM

このうち、forward LSTMは通常のLSTMである。それに対して、backward LSTMひゃ通常のLSTMとは逆に時系列を遡る方向に記憶を行う学習器である。２つのLSTMをそれぞれ逆向きに使うことで双方向に記憶を伝播することが可能となる。
つまり、双方向RNNは二つの方向のRNNを組み合わせた方向となっている。この方式でのRNNの活用例として、時系列データの補完などがあげられる。例えば動画でのフレーム間の画像の補完などが応用例である。

## LSTM/RNNで隠れ状態の最終的なベクトルデータだけでなく学習途中でのデータを出力する

RNNやLSTMでは一般的な手法として隠れ状態をベクトルデータとして内部に保存している。ここで隠れ状態として用いることができるベクトルは次元数がネットワーク構築時に決められており、次元数以上の要素の記憶を行うことが不可能である。そのため、隠れ状態のベクトルは一般的には最終的なベクトルのみが出力される。しかし、隠れ状態のベクトルは学習途中でのデータも含んでいるため、学習途中でのデータを出力することでより良い予測が可能になるのではないかと考えられる。RNNの隠れ状態のベクトルを時系列順にまとめたものをレイヤー間で渡して計算に用いることでより多くの記憶を保存することが可能である。

## 実験内容

今回は、これらの手法を用いた新たなLSTMモデルを用いた関数予測を検討してみる。ここで用いられる関数は以前のレポートで紹介された関数と同一のものである。ここで、次のモデルを試した。

- 双方向lstm(bidirectional_lstm)
- より深くした双方向lstm(deep_bidirectional_lstm)
- より深くしたlstmにdropoutを付加して過学習を抑えたもの(depp_lstm_with_dropout)
- 丸野さんの論文で提案されていたモデル(lstm_with_adam)
- 村田さんの論文で提案されていたモデル(mlp)

また、関数は次のものを試した。

- 村田さんの論文で提案されたf sigma(middle)
- 村田さんの論文で提案されたf(s_x)
- 単純に足し算を行う関数(simple_add)

以下に実験で用いたコードを示す。

In [ ]:
from computable_password_generator import ComputablePasswordGenerator
from utils import Utils
from models import Models
import os
from keras.callbacks import EarlyStopping, TensorBoard

iter = 0
try:
  os.mkdir("outputs/{}".format(iter))
except:
  pass
for model in Models.list_models():
  print("Runnning model: {}".format(model.name))
  for generator in ComputablePasswordGenerator.list_generators():
    early_stopping = EarlyStopping(monitor='val_loss', patience=50)
    tensorboard = TensorBoard(log_dir="logging/{}/".format(iter) + generator.name + "_" + model.name, histogram_freq = 1)
    try:
      print("Testing: generator: {}, model: {}".format(generator.name, model.name))
      print("Figure name: {}".format(generator.name + "_" + model.name))
      generated_passwords = generator.generator(model.required_data_size)
      x_train, x_test, y_train, y_test = Utils.split_to_train_and_valid(generated_passwords)
      x_train = model.resharper(x_train)
      print(x_train.shape)
      x_test = model.resharper(x_test)
      model.model.fit(x_train,y_train,batch_size=model.batch_size, epochs=model.epochs, verbose=1, validation_data=(x_test, y_test), callbacks=[early_stopping, tensorboard])
      history = model.model.history.history
      Utils.plot_history(history, "{}/".format(iter) + generator.name + "_" + model.name)
    except Exception as e:
      print("Error: generator: {}, model: {}".format(generator.name, model.name))
      print(e)
      continue


import numpy as np
import pandas as pd

class ComputablePasswordGenerator:
  # 人間計算可能なパスワードの流出データを模したデータを自動生成する関数群
  # このクラスの関数を実行すると、人間計算可能なパスワードを模したデータが生成され、csvファイルとして保存される
  # 外部のプログラムから呼び出すときは、from computable_password_generator import ComputablePasswordGenerator としてimportを行う

  # この関数群の呼び出しには、int: データ数を引数として与える
  # この関数群はpandas.DataFrameをreturnする
  class Utils:
    @staticmethod
    def sgm(n :int) -> np.ndarray:
      sgm = np.random.randint(0,10,n)
      return sgm

  class GeneratorWithMetadata:
    def __init__(self, generator, name :str):
      self.generator = generator
      self.name = name

  @staticmethod
  def list_generators() -> list:
    generators = []
    generators.append(ComputablePasswordGenerator.GeneratorWithMetadata(ComputablePasswordGenerator.s_x, "s_x"))
    generators.append(ComputablePasswordGenerator.GeneratorWithMetadata(ComputablePasswordGenerator.simple_pointer, "simple_pointer"))
    generators.append(ComputablePasswordGenerator.GeneratorWithMetadata(ComputablePasswordGenerator.password_with_middle, "middle"))
    generators.append(ComputablePasswordGenerator.GeneratorWithMetadata(ComputablePasswordGenerator.password_simple_add, "simple_add"))
    return generators

  @staticmethod
  def password_simple_add(datasize :int) -> np.ndarray:
    result = []
    for row in range(datasize):
      X = ComputablePasswordGenerator.Utils.sgm(14)
      Z = (X[0] + X[1] + X[2]) % 10
      row = np.append(X, Z)
      result.append(row)
    table_array = np.array(result)
    return pd.DataFrame(table_array, columns=["X0", "X1", "X2", "X3", "X4", "X5", "X6", "X7",
      "X8", "X9", "X10", "X11", "X12","X13", "Z"])

  @staticmethod
  def password_with_middle(datasize :int) -> np.ndarray:
    result = []
    for row in range(datasize):
      X = ComputablePasswordGenerator.Utils.sgm(14)
      mid = (X[10] + X[11]) % 10
      Z = (X[mid] + X[12]) % 10
      row = np.append(X, Z)
      result.append(row)
    table_array = np.array(result)
    return pd.DataFrame(table_array, columns=["X0", "X1", "X2", "X3", "X4", "X5", "X6", "X7",
      "X8", "X9", "X10", "X11", "X12","X13", "Z"])

  @staticmethod
  def simple_pointer(datasize :int) -> np.ndarray:
    result = []
    for row in range(datasize):
      X = ComputablePasswordGenerator.Utils.sgm(14)
      mid_1 = (X[10] + X[11]) % 10
      mid_2 = (mid_1 + X[13]) % 10
      mid_3 = (mid_2 + X[12]) % 10
      Z = [mid_3] + X[12]
      row = np.append(X, Z)
      result.append(row)
    table_array = np.array(result)
    return pd.DataFrame(table_array, columns=["X0", "X1", "X2", "X3", "X4", "X5", "X6", "X7",
      "X8", "X9", "X10", "X11", "X12","X13", "Z"])
  
  @staticmethod
  def s_x(datasize :int) -> np.ndarray:
    result = []
    for row in range(datasize):
      X = ComputablePasswordGenerator.Utils.sgm(14)
      S_X = np.zeros(14)
      for k in range(14):
        sgm = ComputablePasswordGenerator.Utils.sgm(14)
        S_X[k] = sgm[X[k]]
      mid = ( S_X[10] + S_X[11] ) % 10
      Z = ( S_X[12] + S_X[13] + S_X[int(mid)] ) %10
      row = np.append(X, Z)
      result.append(row)
    table_array = np.array(result)
    return pd.DataFrame(table_array, columns=["X0", "X1", "X2", "X3", "X4", "X5", "X6", "X7",
      "X8", "X9", "X10", "X11", "X12","X13", "Z"])


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Flatten, Bidirectional
import pandas as pd
import numpy as np

class Models:
  class ModelWithMetadata:
    def __init__(self, model, name :str, batch_size :int, epochs :int, required_data_size :int):
      self.model = model
      self.name = name
      self.batch_size = batch_size
      self.epochs = epochs
      self.required_data_size = required_data_size

    def resharper(self, df :pd.DataFrame) -> (np.ndarray):
      if self.name.find("lstm") != -1:
        print("hoge")
        return df.to_numpy().reshape(df.shape[0], df.shape[1], 1)
      return df

  @staticmethod
  def list_models() -> list:
    models = []
    #models.append(Models.ModelWithMetadata(Models.deep_bidirectional_sequential_lstm_with_dropout(), "deep_bidirectional_lstm_with_dropout_online", 1, 1024, 50000))
    models.append(Models.ModelWithMetadata(Models.deep_bidirectional_sequential_lstm_with_dropout(), "deep_bidirectional_lstm_with_dropout", 32, 4096, 50000))
    models.append(Models.ModelWithMetadata(Models.deep_bidirectional_sequential_lstm(), "deep_bidirectional_lstm", 32, 4096, 50000))
    models.append(Models.ModelWithMetadata(Models.deep_lstm_with_dropout(), "deep_lstm_with_dropout", 32, 4096, 50000))
    models.append(Models.ModelWithMetadata(Models.bidirectional_sequential_lstm(), "bidirectional_lstm", 32, 4096, 50000))
    models.append(Models.ModelWithMetadata(Models.mlp_model(), "mlp", 16, 1024, 10000))
    models.append(Models.ModelWithMetadata(Models.simple_lstm_with_AMSGrad(), "simple_lstm", 32, 1024, 50000))
    models.append(Models.ModelWithMetadata(Models.simple_lstm_with_adam(), "lstm_with_adam", 32, 1024, 50000))
    return models

  # 人間計算可能なパスワードの予測に使うための機械学習モデル群
  # これらの関数を呼び出すと、指定したSequentialモデルがreturnされる
  @staticmethod
  def mlp_model() -> Sequential:
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

  @staticmethod
  def simple_lstm_with_AMSGrad() -> Sequential:
    model = Sequential()
    model.add(LSTM(32, input_shape = (14, 1)))
    model.add(Dense(10, activation="softmax"))
    model.compile(loss="mean_squared_error", optimizer="RMSprop", metrics=["accuracy"])
    return model

  @staticmethod
  def simple_lstm_with_adam() -> Sequential:
    model = Sequential()
    model.add(LSTM(32, input_shape = (14, 1)))
    model.add(Dense(10, activation="softmax"))
    model.compile(loss="mean_squared_error", optimizer="Adam", metrics=["accuracy"])
    return model
  
  @staticmethod
  def deep_lstm_with_dropout() -> Sequential:
    model = Sequential()
    model.add(LSTM(32, input_shape = (14, 1), return_sequences=True, dropout=0.2))
    model.add(LSTM(32, return_sequences=True, dropout=0.2))
    model.add(LSTM(32, dropout=0.2))
    model.add(Dense(10, activation="softmax"))
    model.compile(loss="mean_squared_error", optimizer="Adam", metrics=["accuracy"])
    return model

  @staticmethod
  def bidirectional_sequential_lstm() -> Sequential:
    model = Sequential()
    model.add(Bidirectional(LSTM(32, return_sequences=True), input_shape = (14, 1),))
    model.add(Bidirectional(LSTM(32)))
    model.add(Dense(10, activation="softmax"))
    model.compile(loss="mean_squared_error", optimizer="Adam", metrics=["accuracy"])
    return model
  
  @staticmethod
  def deep_bidirectional_sequential_lstm() -> Sequential:
    model = Sequential()
    model.add(Bidirectional(LSTM(32, return_sequences=True), input_shape = (14, 1), ))
    model.add(Bidirectional(LSTM(32, return_sequences=True)))
    model.add(Bidirectional(LSTM(32, return_sequences=True)))
    model.add(Bidirectional(LSTM(32)))
    model.add(Dense(10, activation="softmax"))
    model.compile(loss="mean_squared_error", optimizer="Adam", metrics=["accuracy"])
    return model
  
  @staticmethod
  def deep_bidirectional_sequential_lstm_with_dropout() -> Sequential:
    model = Sequential()
    model.add(Bidirectional(LSTM(32, return_sequences=True, dropout=0.2), input_shape = (14, 1)))
    model.add(Bidirectional(LSTM(32, return_sequences=True, dropout=0.2)))
    model.add(Bidirectional(LSTM(32, return_sequences=True, dropout=0.2)))
    model.add(Bidirectional(LSTM(32, dropout=0.2)))
    model.add(Dense(10, activation="softmax"))
    model.compile(loss="mean_squared_error", optimizer="Adam", metrics=["accuracy"])
    return model


In [ ]:
from matplotlib import pyplot
from tensorflow import keras
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

class Utils:
  # グラフのプロットを行う関数
  # keras.model.fit.historyおよび画像の保存名を引数として取る
  @staticmethod
  def plot_history(history :str, name :str) -> None:
    # 学習時の学習データおよび正解データに対する正解率を表示する
    pyplot.clf()
    pyplot.plot(history['accuracy'])
    pyplot.plot(history['val_accuracy'])
    pyplot.title('model accuracy')
    pyplot.xlabel('epoch')
    pyplot.ylabel('accuracy')
    pyplot.legend(['acc', 'val_acc'], loc='lower right')
    pyplot.savefig("outputs/" + name + '_accuracy.png')

    # 学習時の学習データおよび正解データに対する損失関数の値を表示する
    pyplot.clf()
    pyplot.plot(history['loss'])
    pyplot.plot(history['val_loss'])
    pyplot.title('model loss')
    pyplot.xlabel('epoch')
    pyplot.ylabel('loss')
    pyplot.legend(['loss', 'val_loss'], loc='lower right')
    pyplot.savefig("outputs/" + name + '_loss.png')
    return None

  @staticmethod
  def split_to_train_and_valid(generated_passwords :pd.DataFrame) -> (np.ndarray, np.ndarray, np.ndarray, np.ndarray):
    generated_passwords = generated_passwords.sample(frac=1).reset_index(drop=True)
    x = generated_passwords.drop(labels = ["Z"],axis = 1)
    y = generated_passwords["Z"]

    # 説明変数・目的変数をそれぞれ訓練データ・テストデータに分割
    x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2)

    # データの型変換
    #x_train = x_train.astype(float)
    #x_test = x_test.astype(float)

    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)

    return x_train, x_test, y_train, y_test


## 実験結果

実験の結果は次のようになった。

### middle関数の場合

- bidirectional_lstmの場合

精度

![middle_bidirectional_lstm_accuracy](../outputs/0/middle_bidirectional_lstm_accuracy.png)

損失関数

![middle_bidirectional_lstm_loss](../outputs/0/middle_bidirectional_lstm_loss.png)

- deep_bidirectional_lstmの場合

精度

![middle_deep_bidirectional_lstm_accuracy](../outputs/0/middle_deep_bidirectional_lstm_accuracy.png)

損失関数

![middle_deep_bidirectional_lstm_loss](../outputs/0/middle_deep_bidirectional_lstm_loss.png)

- deep_lstm_with_dropoutの場合

精度

![middle_deep_lstm_with_dropout_accuracy](../outputs/0/middle_deep_lstm_with_dropout_accuracy.png)

損失関数

![middle_deep_lstm_with_dropout_loss](../outputs/0/middle_deep_lstm_with_dropout_loss.png)

- lstmの場合

精度

![middle_lstm_accuracy](../outputs/0/middle_lstm_with_adam_accuracy.png)

損失関数

![middle_lstm_loss](../outputs/0/middle_lstm_with_adam_loss.png)

- mlpの場合

精度

![middle_mlp_accuracy](../outputs/0/middle_mlp_accuracy.png)

損失関数

![middle_mlp_loss](../outputs/0/middle_mlp_loss.png)

### s_x関数の場合

- bidirectional_lstmの場合

精度

![s_x_bidirectional_lstm_accuracy](../outputs/0/s_x_bidirectional_lstm_accuracy.png)

損失関数

![s_x_bidirectional_lstm_loss](../outputs/0/s_x_bidirectional_lstm_loss.png)

- deep_bidirectional_lstmの場合

精度

![s_x_deep_bidirectional_lstm_accuracy](../outputs/0/s_x_deep_bidirectional_lstm_accuracy.png)

損失関数

![s_x_deep_bidirectional_lstm_loss](../outputs/0/s_x_deep_bidirectional_lstm_loss.png)

- deep_lstm_with_dropoutの場合

精度

![s_x_deep_lstm_with_dropout_accuracy](../outputs/0/s_x_deep_lstm_with_dropout_accuracy.png)

損失関数

![s_x_deep_lstm_with_dropout_loss](../outputs/0/s_x_deep_lstm_with_dropout_loss.png)

- lstmの場合

精度

![s_x_lstm_accuracy](../outputs/0/s_x_lstm_with_adam_accuracy.png)

損失関数

![s_x_lstm_loss](../outputs/0/s_x_lstm_with_adam_loss.png)

- mlpの場合

精度

![s_x_mlp_accuracy](../outputs/0/s_x_mlp_accuracy.png)

損失関数

![s_x_mlp_loss](../outputs/0/s_x_mlp_loss.png)

### simple_add関数の場合

- bidirectional_lstmの場合

精度

![simple_add_bidirectional_lstm_accuracy](../outputs/0/simple_add_bidirectional_lstm_accuracy.png)

損失関数

![simple_add_bidirectional_lstm_loss](../outputs/0/simple_add_bidirectional_lstm_loss.png)

- deep_bidirectional_lstmの場合

精度

![simple_add_deep_bidirectional_lstm_accuracy](../outputs/0/simple_add_deep_bidirectional_lstm_accuracy.png)

損失関数

![simple_add_deep_bidirectional_lstm_loss](../outputs/0/simple_add_deep_bidirectional_lstm_loss.png)

- deep_lstm_with_dropoutの場合

精度

![simple_add_deep_lstm_with_dropout_accuracy](../outputs/0/simple_add_deep_lstm_with_dropout_accuracy.png)

損失関数

![simple_add_deep_lstm_with_dropout_loss](../outputs/0/simple_add_deep_lstm_with_dropout_loss.png)

- lstmの場合

精度

![simple_add_lstm_accuracy](../outputs/0/simple_add_lstm_with_adam_accuracy.png)

損失関数

![simple_add_lstm_loss](../outputs/0/simple_add_lstm_with_adam_loss.png)

- mlpの場合

精度

![simple_add_mlp_accuracy](../outputs/0/simple_add_mlp_accuracy.png)

損失関数

![simple_add_mlp_loss](../outputs/0/simple_add_mlp_loss.png)


## 考察

双方向RNNを用いることで、狙い通りに村田さんが提案した関数に対する正解率が既存のRNNと比較して向上した。具体的な数値としては、既存の研究では13%ほどであった正解率が50%程度まで向上していることが見て取れる。一方で、まだ正解率が15%程度であることから、依然として機械学習を用いたセキュリティの突破は現実的ではない。また、単純な足し算関数に関しては村田さんの論文でも50%程度正解できるとの結果が出ていたが、本手法を用いることにより正解率をほぼ100%まで向上させることができた。今後は他のLSTM手法および複雑なモデルによる突破も検討する。